In [16]:
from ultralytics import YOLO
import cv2
import numpy as np


front_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg'
back_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg'
model_path = r'C:\Users\saart\OneDrive\Desktop\best.pt'
model = YOLO(model_path)

def predict(image_path):
    results = model.predict(image_path, save=True)
    fruits = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        class_name = model.names[int(box.cls)]        
        # Calculate center point
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        
        fruits.append({
            "bbox": (x1, y1, x2, y2),
            "center": (center_x, center_y),
            "class": class_name
        })
    
    return fruits

# Step 3: Simple color detection (RGB-based)
def get_color(roi):
    avg_color = np.mean(roi, axis=(0, 1))  # Get average RGB values
    r, g, b = avg_color
    
    # Simple color thresholds (adjust these as needed)
    if r > g and r > b: return "red"
    elif g > r and g > b: return "green"
    elif b > r and b > g: return "blue"
    else: return "unknown"

# Step 4: Process images
front_img = cv2.imread(front_image)
back_img = cv2.imread(back_image)

front_fruits = predict(front_image)
back_fruits = predict(back_image)

# Step 5: Match fruits between front and back
counts = {"red": 0, "green": 0, "blue": 0, "unknown": 0}

# Check front fruits
for front in front_fruits:
    x1, y1, x2, y2 = front["bbox"]
    roi = front_img[y1:y2, x1:x2]
    color = get_color(roi)
    
    # Check if same fruit exists in back image
    is_duplicate = False
    for back in back_fruits:
        # Simple distance check (adjust 50 as needed)
        distance = np.sqrt((front["center"][0]-back["center"][0])**2 + 
                          (front["center"][1]-back["center"][1])**2)
        
        if distance < 50:  # If centers are close
            is_duplicate = True
            break
    
    if not is_duplicate:
        counts[color] += 1

# Check remaining back fruits
for back in back_fruits:
    x1, y1, x2, y2 = back["bbox"]
    roi = back_img[y1:y2, x1:x2]
    color = get_color(roi)
    counts[color] += 1

# Step 6: Show results
print(front_fruits)
print("Final counts:")
for color, count in counts.items():
    print(f"{color}: {count}")


image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg: 480x640 5 2s, 3 3s, 291.1ms
Speed: 3.0ms preprocess, 291.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict15

image 1/1 C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg: 480x640 8 2s, 3 3s, 215.0ms
Speed: 2.0ms preprocess, 215.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict15
[{'bbox': (235, 211, 374, 336), 'center': (304, 273), 'class': '3'}, {'bbox': (25, 211, 167, 337), 'center': (96, 274), 'class': '3'}, {'bbox': (441, 212, 581, 335), 'center': (511, 273), 'class': '3'}, {'bbox': (68, 236, 86, 277), 'center': (77, 256), 'class': '2'}, {'bbox': (278, 236, 295, 276), 'center': (286, 256), 'class': '2'}, {'bbox': (319, 272, 336, 312), 'center': (327, 292), 'class': '2'}, {'bbox': (488, 287, 505, 324), 'center': (496, 305), 'class': '2'}, {'bbox': (109, 272, 127, 313), 'cen

In [4]:
import shutil

# Zip the entire runs directory
shutil.make_archive('runs', 'zip', 'runs')
print("Runs directory zipped as runs.zip")
from IPython.display import FileLink

# Create a download link for the zipped file
FileLink('runs.zip')


Runs directory zipped as runs.zip


C:\Users\saart\runs.zip